# Section-1: Import Libraries

In [14]:
import requests # This library helps us to fetch data from API
import pandas as pd # For handling and analysing data
import numpy as np # For numerical operations
from sklearn.model_selection import train_test_split # To split data into training and testing sets
from sklearn.preprocessing import LabelEncoder # TO convert catogerical data into numericals values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor # Models for classification and regression tasks
from sklearn.linear_model import LinearRegression  # Linear regression model for classification
from sklearn.metrics import mean_squared_error # To measure the  accuracy of out prediction
from datetime import datetime, timedelta # To handle date & time
from sklearn.metrics import mean_squared_error, classification_report  # Metrics
import pytz

In [15]:
API_KEY = "7ec3ca4280bb09b11fab0b2153721c10"
BASE_URL = "https://api.openweathermap.org/data/2.5/" # for making api request

# 1.Fetch Weather Data

In [16]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" # construct the API request URL
    response = requests.get(url) # send the get request to API
    data = response.json()

    # return a dictionary
    return {
        "city": data["name"],
        "current_temp": round(data["main"]["temp"]),
        "feels_like": round(data["main"]["feels_like"]),
        "temp_min": round(data["main"]["temp_min"]),
        "temp_max": round(data["main"]["temp_max"]),
        "humidity": round(data["main"]["humidity"]),
        "description": data["weather"][0]["description"],
        "country": data["sys"]["country"],
        "wind_gust_dir": data["wind"]["deg"],
        "pressure": data["main"]["pressure"],
        "wind_gust_speed": data["wind"]["speed"]
    }

# 2.Read Historical Data

In [17]:
def read_historical_data(filename):
    df = pd.read_csv(filename) # load csv file into dataFrame
    df = df.dropna() # remove rows with missing values
    df = df.drop_duplicates()
    return df

# 3.Prepare Data For Training

In [18]:
def prepare_data(data):
    le = LabelEncoder()  # initialize LabelEncoder instance

    # Convert categorical data to numerical
    data["WindGustDir"] = le.fit_transform(data["WindGustDir"])
    data["RainTomorrow"] = le.fit_transform(data["RainTomorrow"])

    # Define the feature variables and target variable
    X = data.iloc[:,:-1]  # feature variables
    y = data.iloc[:,-1]  # target variable

    return X, y, le

# 4.Train Rain Prediction Model

In [19]:
# Function to train the Rain Prediction model with RandomForest
def train_rain_model_rf(X,y):
    """
    This function train our model using classification to predict data
    
    * random_state: reproducibility making sure results are consistent
    * n_estimators: to use 100 decision tree
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
    model_rf.fit(X_train, y_train)
    y_pred_rf = model_rf.predict(X_test)
    
    # Metrics for RandomForest model
    print("RandomForest Rain Prediction Metrics:")
    print("\n", classification_report(y_test, y_pred_rf))
    print("\nMean Square Error for Rain Model")
    print(mean_squared_error(y_test, y_pred_rf))
    
    return model_rf

In [20]:
# Function to train the Rain Prediction model with LinearRegression
def train_rain_model_lr(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model_lr = LinearRegression()
    model_lr.fit(X_train, y_train)
    y_pred_lr = (model_lr.predict(X_test) > 0.5).astype(int) # Convert to binary predictions
    
    # Metrics for LinearRegression model
    print("\nLinearRegression Rain Prediction Metrics:")
    print("\n", classification_report(y_test, y_pred_lr))
    print("\nMean Square Error for Rain Model")
    print(mean_squared_error(y_test, y_pred_lr))    
    
    return model_lr

In [21]:
# Comparison of Rain Prediction Models
def compare_rain_prediction_models(X,y, le):
    
    # Train both models and display metrics
    rf_model = train_rain_model_rf(X, y)
    lr_model = train_rain_model_lr(X, y)
    print("\n")
    
    return rf_model, lr_model

# 5.Prepare regression data

In [22]:
def prepare_regression_data(dataFrame, feature):
    """
    To predict future values of a feature (like temperature & humidity), we need to use regression.
    Regression helps us predict continuous values, such as numerical measurements.
    """
    
    # Initialize lists to store feature (X) and target (y) values
    X, y = [], []

    # Iterate over the data frame rows, excluding the last one
    for i in range(len(dataFrame) - 1):
        # Append the current value of the specified feature to X
        X.append(dataFrame[feature].iloc[i])
        # Append the next value of the specified feature to y, creating a target value for each input
        y.append(dataFrame[feature].iloc[i + 1])

    # Convert X to a 2D numpy array and reshape it to be compatible with scikit-learn (each value is an array)
    X = np.array(X).reshape(-1, 1)
    # Convert y to a numpy array for consistency in data format
    y = np.array(y)

    # Return the prepared feature and target arrays
    return X, y

## 6.Train Regression Model

In [23]:
def train_regression_model(X, y):
    # Initialize a RandomForestRegressor model with 100 decision trees
    # and a fixed random seed for reproducibility.
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # Train the regression model on the feature set X and target y
    model.fit(X, y)

    # Return the trained model for later use in making predictions
    return model

## 7.Predict Future

In [24]:
def predict_future(model, current_value):
    """
    Predict future temperature values.
    
    model: The trained RandomForestRegressor model
    current_value: The latest temperature value we have
    """
    
    # Start predictions list with the most recent temperature
    predictions = [current_value] 

    # Predict the next 5 values (for the next 5 hours in this example)
    for i in range(5):
        # Predict the next temperature based on the latest predicted value
        next_value = model.predict([[predictions[-1]]])  # Use last prediction as input
        
        # Add this predicted temperature to the list
        predictions.append(next_value[0])

    # Return only the future predictions (skip the first element)
    return predictions[1:]

# 8.Weather Analysis Function

In [25]:
# Weather Analysis Function

def weather_view():
    # Prompt the user to enter a city name
    city = input("Enter any city name: ")
    
    # Fetch the current weather for the entered city
    current_weather = get_current_weather(city)

    # Load historical weather data from a CSV file
    historical_data = read_historical_data("./data/weather.csv")

    # Prepare data and train the rain prediction model
    X, y, le = prepare_data(historical_data)  # X, y are features and labels, le is the label encoder
    # Compare RandomForest and LinearRegression models
    rf_model, lr_model = compare_rain_prediction_models(X,y,le)
    rain_model = lr_model # Train model for rain prediction

    # Map wind direction (in degrees) to compass points (N, NE, E, etc.)
    wind_deg = current_weather['wind_gust_dir'] % 360  # Normalize wind degree to a 0-360 range
    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
    ]
    
    # Find the compass direction based on the wind degree
    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
    
    # Encode compass direction using the label encoder (le), if it exists in the classes
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    # Prepare the data for the current weather to make predictions
    current_data = {
        "MinTemp": current_weather["temp_min"],
        "MaxTemp": current_weather["temp_max"],
        "WindGustDir": compass_direction_encoded,
        "WindGustSpeed": current_weather["wind_gust_speed"],
        "Humidity": current_weather["humidity"],
        "Pressure": current_weather["pressure"],
        "Temp": current_weather["current_temp"]
    }

    # Convert current data into a DataFrame for model prediction
    current_df = pd.DataFrame([current_data])

    # Predict rain based on the current data
    rain_prediction = rain_model.predict(current_df)[0]

    # Prepare regression data for temperature and humidity
    X_temp, y_temp = prepare_regression_data(historical_data, "Temp")  # Features and target for temperature
    X_hum, y_hum = prepare_regression_data(historical_data, "Humidity")  # Features and target for humidity

    # Train regression models to predict temperature and humidity
    temp_model = train_regression_model(X_temp, y_temp)
    humidity_model = train_regression_model(X_hum, y_hum)

    # Predict future temperature and humidity values
    future_temp = predict_future(temp_model, current_weather["temp_min"])
    future_humidity = predict_future(humidity_model, current_weather["humidity"])

    # Prepare timestamps for the next 5 hours for future predictions
    timezone = pytz.timezone("Asia/Dhaka")  # Set the timezone
    now = datetime.now(timezone)  # Get the current time
    next_hour = now + timedelta(hours=1)  # Move to the next hour
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)  # Round to the hour

    # Generate a list of time labels for the next 5 hours
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # Display the results
    print(f"City: {city}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}°C")
    print(f"Feels Like: {current_weather['feels_like']}°C")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    # Display future temperature predictions
    print("\nFuture Temperature:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    # Display future humidity predictions
    print("\nFuture Humidity:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

In [26]:
weather_view()

Enter any city name:  Dhaka


RandomForest Rain Prediction Metrics:

               precision    recall  f1-score   support

           0       0.85      0.98      0.91        57
           1       0.86      0.38      0.52        16

    accuracy                           0.85        73
   macro avg       0.85      0.68      0.72        73
weighted avg       0.85      0.85      0.83        73


Mean Square Error for Rain Model
0.1506849315068493

LinearRegression Rain Prediction Metrics:

               precision    recall  f1-score   support

           0       0.85      1.00      0.92        57
           1       1.00      0.38      0.55        16

    accuracy                           0.86        73
   macro avg       0.93      0.69      0.73        73
weighted avg       0.88      0.86      0.84        73


Mean Square Error for Rain Model
0.136986301369863


City: Dhaka, BD
Current Temperature: 30°C
Feels Like: 32°C
Minimum Temperature: 30°C
Maximum Temperature: 30°C
Humidity: 54%
Weather Prediction: haze
Rain